In [1]:
!conda install matplotlib --yes
!conda install torchvision --yes
!conda install torch --yes

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: unsuccessful initial attempt using frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - torch

Current channels:

  - https://repo.anaconda.com/pkgs/main/osx-arm64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/osx-arm64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [2]:
import numpy as np 
import pandas as pd 
import torch
from torch import nn 
from torch.nn import functional as F
import torch.optim as optim
from PIL import Image 

from matplotlib import pyplot as plt

import torchvision.transforms as transforms
import torchvision.models as models

import copy
import os 
import glob

In [3]:
# observando disponibilidade do cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
import sys 
sys.setrecursionlimit(10000)

In [5]:
# residual network from convolutional neural networks 

class ResBlock(nn.Module): 

    def __init__(self, in_channels, out_channels):
        super(ResBlock, self).__init__()
        
        #defining 3 convolutional layers 2D 
        self.conv1 = nn.Conv2d(in_channels,16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16,32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32,64, kernel_size=3, stride=1, padding=1)

        #defining average pooling (or maxpool) for the layer
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)
        
        #defining the layer 2D
        self.convOut = nn.Conv2d(64,out_channels, kernel_size=1)
        
    def __call__(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)

        x = self.pool1(x)
        x = self.convOut(x)

        return x

In [6]:
#script phase 

block = ResBlock(3, 16)
X = np.random.randn(3, 512, 8)
X = np.float32(X)
print(X.shape, type(X))

x = torch.from_numpy(X)
x = x.unsqueeze(0)

out = block(x)
print(out.shape)

(3, 512, 8) <class 'numpy.ndarray'>
torch.Size([1, 16, 256, 4])


In [7]:
# to the image output 
#TODO: as imagens podem sair 1024 ou 512 pq temos gpu 
def ImageSeg(image):
    imageSize = 1024 if torch.cuda.is_available() else 512

    loader = transforms.Compose([
    transforms.Resize(imageSize), 
    transforms.ToTensor])

    imageLoad = Image.open(image)
    imageLoad = loader(imageLoad).unsqueeze(0)
    return imageLoad.to(device, torch.float)
        

In [8]:
#agora faz o contrario 
def TensorToImage(tensor, title=None): 
    unloader = transforms.ToPILImage()
    plt.ion()

    image = tensor.cpu().clone()
    image = image.squeeze(0)
    imageRes = unloader(image)

    plt.imshow(imageRes)

    if title is not None: 
        plt.title(title)
    plt.pause(0.001)


In [9]:
#loss function 

class ContentLoss(nn.Module):
    
    def __init__(self, target): 
        super(ContentLoss, self).__init__
        self.target = target.detach()
        
    def forward(self, input): 
        self.loss = F.mse_loss(input, self.target)
        return input


In [10]:
def gramMatrix(input): 
        a, b, c, d = input.size()
        #a batch size 
        #b = number of feature maps 
        #c and d are dimensions of a feature map 
        features = input.view(a * b, c* d)
        
        G= torch.mm(features, features.t())
        return G.div(a * b * c * d)

In [11]:
class StyleLoss(nn.Module): 
    
    def __init__(self, targetFeature): 
        super(StyleLoss, self).__init__()
        self.targetFeature = gramMatrix(targetFeature).detach()
        
    def forward(self, input): 
        G = gramMatrix(input)
        self.loss = F.mse_loss(G, self.target)
        return input 

In [12]:
cnn = models.vgg19(pretrained=True).features.eval()

/Users/anakarolina/anaconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Users/anakarolina/anaconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406])
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225])

class Normalization(nn.Module): 
    def __init__(sefl, mean, std):
        super(Normalization, self).__init__()
        self.mean = torch.tensor(mean).view(-1, 1, 1)
        self.std = torch.tensor(std).view(-1, 1, 1)
    
    def forward(self, img):
        return (img - self.mean) / self.std

In [14]:
content_layers_default = ['conv_4']
style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

def getStyleModelAndLoss(cnn, normalizationMean, normalizationStd, styleImg, contentImg, contentLayers=content_layers_default, styleLayers=style_layers_default):
    normalization = Normalization(normalizationMean, normalizationStd)
    contentLosses = []
    styleLosses = []
    
    model = nn.Sequential(normalization)
    i = 0 
    
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i+=1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'batchn_{}'.format(i)
        else: 
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))
        
    model.add_module(name, layer)
    
    if name in contentLayers:
        target = model(contentImg).detach()
        contentLoss = ContentLoss(target)
        model.add_module("content_loss_{}".format(i), contentLoss)
        contentLosses.append(contentLoss)
    
    if name in styleLayers: 
        targetFeature = model(styleImg).detach()
        styleLoss = StyleLoss(targetFeature)
        model.add_module("style_loss_{}".format(i), styleLoss)
        styleLosses.append(styleLoss)
        
    for i in ragen(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break 
        model = model[:(i + 1)]
        
    return model, styleLosses, contentLosses

In [15]:
content_layers_default = ['conv_4']
style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

def getStyleModelAndLosses(cnn, normalizationMean, normalizationStd, styleImg, contentImg, contentLayers=content_layers_default, styleLayers=style_layers_default):
    normalization = Normalization(normalizationMean, normalizationStd)
    contentLosses = []
    styleLosses = []
    
    model = nn.Sequential(normalization)
    i = 0 
    
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i+=1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'batchn_{}'.format(i)
        else: 
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))
        
    model.add_module(name, layer)
    
    if name in contentLayers:
        target = model(contentImg).detach()
        contentLoss = ContentLoss(target)
        model.add_module("content_loss_{}".format(i), contentLoss)
        contentLosses.append(contentLoss)
    
    if name in styleLayers: 
        targetFeature = model(styleImg).detach()
        styleLoss = StyleLoss(targetFeature)
        model.add_module("style_loss_{}".format(i), styleLoss)
        styleLosses.append(styleLoss)
        
    for i in ragen(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break 
        model = model[:(i + 1)]
        
    return model, styleLosses, contentLosses

In [16]:
#copy of the content image here 

In [17]:
def getInputOptimizer(inputImage):
    optimizer = optim.LBFGS([inputImage])
    return optimizer

def runStyleTranser(cnn, normalizationMean, normalizationStd, contentImg, styleImg, inputImage, numSteps=300, styleWeight=1000000, contentWeight=1):
    print("building the style transfer")
    model, styleLosses, contentLosses = getStyleModelAndLosses(cnn, normalizationMean, normalizationStd, styleImg, contentImg)
    
    inputImage.requires_grad(True)
    model.eval()
    model.requires_grad(False)
    
    optimizer = getInputOptimizer(inputImage)
    print("Optimizing...")
    run = [0]
    while run[0] <= numSetps: 
        
        def closure(): 
            with torch.no_grad():
                inputImg.clamp_(0, 1)
            optimizer.zero_grad()
            model(inputImage)
            styleScore = 0
            contentScore = 0
            
            for sl in styleLosses: 
                styleScore += sl.loss
            for cl in contentLosses: 
                contentScore += cl.loss
                
            styleScore *= styleWeight
            contentScore *= contentWeight
            
            loss = styleScore + contentScore
            loss.backward()
            
            run[0] += 1
            if run[0] % 50 == 0:
                print("run {}:".format(run))
                print('Style Loss: {:4f}'.format(styleScore.item(), contentScore.item()))
                print()
                
            return styleScore + contentScore
        optimizer.step(closure)
    with torch.no_grad():
        inputImage.clamp_(0, 1)
    return inputImage

In [20]:
imageStyle = []
for filename in glob.glob('Documents/Ghibli(paisagens)/*.png'):
    im=Image.open(filename)
    plt.figure()
    plt.imshow(im) 
    plt.show(im)
    imageForProccess = ImageSeg(im)
    imageStyle.append(imageForProccess)

#print(imageStyle)